# 📚 Book Fixer Worker

This notebook fixes books using free LLM APIs. Run all cells to start processing.

**Setup:**
1. Set your API keys in the cell below
2. Paste the work URL from the coordinator
3. Run all cells!


In [ ]:
#@title 🔑 API Keys Configuration
#@markdown Get free API keys from:
#@markdown - [Groq](https://console.groq.com) - Fast, free tier
#@markdown - [Together](https://api.together.xyz) - Free credits
#@markdown - [Cerebras](https://cerebras.ai) - Free inference
#@markdown - [OpenRouter](https://openrouter.ai) - Free models
#@markdown - [DeepSeek](https://deepseek.com) - Very cheap

import os

GROQ_API_KEY = "" #@param {type:"string"}
TOGETHER_KEY = "" #@param {type:"string"}
CEREBRAS_API_KEY = "" #@param {type:"string"}
OPENROUTER_KEY = "" #@param {type:"string"}
DEEPSEEK_API_KEY = "" #@param {type:"string"}
FIREWORKS_API_KEY = "" #@param {type:"string"}

# Set environment variables
os.environ['GROQ_API_KEY'] = GROQ_API_KEY
os.environ['TOGETHER_KEY'] = TOGETHER_KEY
os.environ['CEREBRAS_API_KEY'] = CEREBRAS_API_KEY
os.environ['OPENROUTER_KEY'] = OPENROUTER_KEY
os.environ['DEEPSEEK_API_KEY'] = DEEPSEEK_API_KEY
os.environ['FIREWORKS_API_KEY'] = FIREWORKS_API_KEY

print("✅ API keys configured!")

In [ ]:
#@title 📥 Work Queue URL
#@markdown Paste the work queue URL from the coordinator

WORK_URL = "" #@param {type:"string"}
MAX_BOOKS = 10 #@param {type:"integer"}

print(f"Will process up to {MAX_BOOKS} books")

In [ ]:
#@title 🔧 Worker Code
import json
import time
import random
import requests
from pathlib import Path

APIS = [
    ("Groq", "https://api.groq.com/openai/v1/chat/completions",
     os.environ.get('GROQ_API_KEY', ''), "llama-3.3-70b-versatile"),
    ("Cerebras", "https://api.cerebras.ai/v1/chat/completions",
     os.environ.get('CEREBRAS_API_KEY', ''), "llama3.1-70b"),
    ("Together", "https://api.together.xyz/v1/chat/completions",
     os.environ.get('TOGETHER_KEY', ''), "meta-llama/Llama-3.3-70B-Instruct-Turbo"),
    ("Fireworks", "https://api.fireworks.ai/inference/v1/chat/completions",
     os.environ.get('FIREWORKS_API_KEY', ''), "accounts/fireworks/models/llama-v3p3-70b-instruct"),
    ("OpenRouter", "https://openrouter.ai/api/v1/chat/completions",
     os.environ.get('OPENROUTER_KEY', ''), "meta-llama/llama-3.3-70b-instruct"),
    ("DeepSeek", "https://api.deepseek.com/v1/chat/completions",
     os.environ.get('DEEPSEEK_API_KEY', ''), "deepseek-chat"),
]

_api_idx = random.randint(0, len(APIS) - 1)

def call_llm(prompt, max_tokens=4000, temperature=0.7):
    global _api_idx
    for i in range(len(APIS)):
        idx = (_api_idx + i) % len(APIS)
        name, url, key, model = APIS[idx]
        if not key:
            continue
        try:
            headers = {"Authorization": f"Bearer {key}", "Content-Type": "application/json"}
            if "openrouter" in url:
                headers["HTTP-Referer"] = "https://colab.research.google.com"
            resp = requests.post(url, headers=headers,
                json={"model": model, "messages": [{"role": "user", "content": prompt}],
                      "max_tokens": max_tokens, "temperature": temperature},
                timeout=120)
            if resp.status_code == 200:
                print(f"  ✓ {name}")
                _api_idx = (idx + 1) % len(APIS)
                return resp.json()["choices"][0]["message"]["content"]
            elif resp.status_code == 429:
                print(f"  ⚠ {name} rate limited")
                time.sleep(2)
        except Exception as e:
            print(f"  ✗ {name}: {str(e)[:50]}")
    return None

def analyze_names(chapters):
    all_text = "\n".join(chapters)
    prompt = f"""Analyze this story for character name inconsistencies.
Look for same character with different names/spellings.

Text (first 8000 chars):
{all_text[:8000]}

Return JSON array:
[{{"character": "main name", "variants": ["var1"], "fix_to": "correct"}}]

Return ONLY JSON array."""
    result = call_llm(prompt, max_tokens=1000)
    if result:
        try:
            start, end = result.find('['), result.rfind(']') + 1
            if start >= 0 and end > start:
                return json.loads(result[start:end])
        except: pass
    return []

def expand_chapter(text, chapter_num):
    word_count = len(text.split())
    target = max(3000, word_count + 1000)
    prompt = f"""Expand this chapter to {target} words. Add sensory details, character thoughts, dialogue.
Keep same plot and tone.

Chapter {chapter_num} ({word_count} words):
{text}

Write expanded chapter:"""
    result = call_llm(prompt, max_tokens=6000)
    return result if result and len(result.split()) > word_count else text

def fix_book(book_data):
    print(f"\n{'='*50}\nFixing: {book_data['name']}\n{'='*50}")
    chapters = book_data.get("chapters", {})
    if not chapters:
        return None
    
    chapter_list = [chapters[f"chapter_{i:02d}"] for i in range(1, 20) if f"chapter_{i:02d}" in chapters]
    if not chapter_list:
        return None
    
    print("\n[1/3] Analyzing names...")
    name_fixes = analyze_names(chapter_list)
    print(f"  Found {len(name_fixes)} issues")
    
    print("\n[2/3] Applying fixes...")
    fixed = {}
    for i, text in enumerate(chapter_list, 1):
        key = f"chapter_{i:02d}"
        for fix in name_fixes:
            for var in fix.get("variants", []):
                text = text.replace(var, fix.get("fix_to", ""))
        fixed[key] = text
    
    print("\n[3/3] Expanding short chapters...")
    for key, text in fixed.items():
        if len(text.split()) < 2500:
            print(f"  {key}: {len(text.split())} words")
            fixed[key] = expand_chapter(text, key)
            print(f"    → {len(fixed[key].split())} words")
    
    return {"name": book_data["name"], "chapters": fixed, "fixes": name_fixes}

print("✅ Worker code loaded!")
active = [n for n, _, k, _ in APIS if k]
print(f"Active APIs: {', '.join(active) or 'NONE - set keys above!'}")

In [ ]:
#@title 🚀 Run Worker

def get_work(url):
    if url:
        try:
            resp = requests.get(url, timeout=30)
            if resp.status_code == 200:
                return resp.json()
        except Exception as e:
            print(f"Failed to get work: {e}")
    
    # Demo data if no URL
    print("Using demo data (no work URL provided)")
    return [{"name": "Demo_Book", "chapters": {
        "chapter_01": "John walked in. Jon looked around. Jonn felt scared. The room was dark.",
        "chapter_02": "John found a door. It led somewhere mysterious."
    }}]

def save_results(results):
    out = Path("fixed_books")
    out.mkdir(exist_ok=True)
    for r in results:
        if not r: continue
        d = out / r["name"]
        d.mkdir(exist_ok=True)
        for k, v in r["chapters"].items():
            (d / f"{k}.md").write_text(v)
        (d / "fixes.json").write_text(json.dumps(r.get("fixes", []), indent=2))
        print(f"Saved: {r['name']}")

# Main execution
print("="*60)
print("📚 BOOK FIXER WORKER")
print("="*60)

books = get_work(WORK_URL)
results = []

for i, book in enumerate(books[:MAX_BOOKS]):
    print(f"\n[{i+1}/{min(len(books), MAX_BOOKS)}]")
    result = fix_book(book)
    if result:
        results.append(result)

if results:
    save_results(results)
    print(f"\n✅ Fixed {len(results)} books!")
    print("📁 Download 'fixed_books' folder from the file browser (left panel)")

In [ ]:
#@title 📦 Download Results (zip)
import shutil

if Path("fixed_books").exists():
    shutil.make_archive("fixed_books", "zip", "fixed_books")
    print("✅ Created fixed_books.zip")
    print("Click the folder icon on the left, then download fixed_books.zip")
    
    # For Colab, trigger download
    try:
        from google.colab import files
        files.download("fixed_books.zip")
    except:
        pass
else:
    print("No results to download. Run the worker first!")